# 🚀 Notebook 4 — On‑prem Deploy with Docker Compose + STS (Phase 1 stack)

> **Scope:** we’ll deploy the **TaskFlow Phase 1** stack (Gateway + tasks‑service) **on‑prem**.  
> We’ll use **Docker Compose** (recommended) and also show how to run from **STS** for development.  
> **Not in scope now:** AWS/Azure/Kubernetes (those come later).

---

## 🎯 Goals

- Bring up the **Phase 1** stack (Gateway **8080** → tasks‑service **8081**) on a **local or remote on‑prem** host.
- Use **Docker Compose (`run_compose.sh`)** or **STS** to run services.
- Configure **optional X‑API‑KEY** via environment variable and validate routing `/api/tasks/**`.
- Check **observability** with `/actuator/health` and basic logs.
- Understand **what is safe to expose** publicly (and what is not) in this MVP.

---

## 🧭 Context recap (what we’re deploying)

- **Gateway (Spring Cloud Gateway)** on **port 8080**, routing **/api/tasks/** → `tasks-service`.
- **tasks‑service (Spring Boot)** on **port 8081**, H2 in‑memory DB (data is not persisted across restarts).
- **Optional security:** header `X-API-KEY` enforced **only if** env var `API_KEY` is defined for the gateway.
- **Observability:** `/actuator/health` on both services.

> This is a **simple on‑prem** setup for demos/learning. **Production‑grade** topics (TLS, JWT/IAM, persistent DB) arrive in later modules.

---

## 🧰 Prerequisites

- **On‑prem host** (local laptop/desktop or a remote Linux server) with:
  - **Docker** and **Docker Compose**
  - Network access to the ports you’ll use (default **8080**, **8081**)
- **Git**, **Java 21**, **Maven** if you also plan to use **STS** locally
- **Postman** (recommended) for tests — `curl` is optional

---

## 1) Get the code (Phase 1)

```bash
git clone https://github.com/smartlearningci/cloud_java
cd cloud_java
git checkout phase-1
```

**What’s included in Phase 1:**
- `tasks-service` (Spring Boot, H2, Actuator) listening on **8081**
- `gateway` (Spring Cloud Gateway) listening on **8080**, forwarding `/api/tasks/**` to the backend
- Optional **X‑API‑KEY** gate (enabled only when `API_KEY` is set in the gateway process environment)

---

## 2) Deploy with **Docker Compose** (recommended)

> Use this when running on your **on‑prem host** (local or remote). It brings up **both** services together.

### 2.1 (Optional) Enable **X‑API‑KEY**
Choose **ONE** method:
- **Session environment variable** (bash):
  ```bash
  export API_KEY=supersecret
  ```
- **`.env` file** at the project root:
  ```
  API_KEY=supersecret
  ```

> If you **do not** set `API_KEY`, the gateway runs in **open mode** for dev/demos (no header required).

### 2.2 Start the stack
```bash
./run_compose.sh
# (alternative if you prefer)
# docker compose up --build
```

### 2.3 Verify health of both services
- Gateway: `http://<host>:8080/actuator/health` → expect `{"status":"UP"}`
- Backend: `http://<host>:8081/actuator/health` → expect `{"status":"UP"}`

### 2.4 Test routing via the gateway
- List: `GET http://<host>:8080/api/tasks`  
- Create: `POST http://<host>:8080/api/tasks` (JSON body)  
- Update status: `PATCH http://<host>:8080/api/tasks/{id}/status`

If `API_KEY` was set, include header: `X-API-KEY: <your value>`.

> 💡 **Logs**: use `docker compose logs -f gateway` and `docker compose logs -f tasks-service` to watch runtime logs.

### 2.5 Stop / restart
```bash
# stop (keep containers)
docker compose stop

# restart (already built)
docker compose start

# rebuild & restart from scratch
docker compose down
docker compose up --build
```

---

## 3) Run from **STS** (development path)

This is useful to debug or when you prefer running from the IDE.

### 3.1 Start `tasks-service` (8081)
- STS → **File > Import > Maven > Existing Maven Projects** → select the `tasks-service` project
- Open main class (e.g., `…TasksServiceApplication`)
- **Run As > Spring Boot App**

### 3.2 Start `gateway` (8080)
- Import the `gateway` project in STS
- To require `X-API-KEY`, set env var `API_KEY` in **Run Configuration** of the gateway
- **Run As > Spring Boot App**

> For demos on a LAN, you can call `http://<dev-machine-ip>:8080/api/tasks`. **Do not** expose IDE‑run processes to the public Internet.

---

## 4) Testing (Postman recommended)

### 4.1 Environment variables
- `baseUrl = http://<host>:8080/api`
- `apiKey = supersecret` (only if you enabled `API_KEY`)

### 4.2 Requests
1) **List**
   - `GET {{baseUrl}}/tasks`
   - If `API_KEY` is enabled: Header `X-API-KEY: {{apiKey}}`
2) **Create**
   - `POST {{baseUrl}}/tasks`
   - Headers: `Content-Type: application/json` (+ `X-API-KEY` if enabled)
   - Body example:
```json
{
  "title": "On-prem demo",
  "description": "Notebook 4",
  "projectId": "OPS",
  "assignee": "Team"
}
```
3) **Update status**
   - `PATCH {{baseUrl}}/tasks/{{taskId}}/status`
   - Body:
```json
{ "status": "DOING" }
```

> If you prefer terminal, replicate the same with `curl`. Postman is friendlier for the cohort and easy to share collections.

---

## 5) Safe exposure checklist (if you plan to reach it from outside)

> MVP warning: in **Phase 1** there is **no TLS** and **no real user auth**. If you put this on the public Internet, do it **only** for short demos and **with X‑API‑KEY enabled**.

- [ ] Use a **non‑default** API key (env var) and rotate it if shared.  
- [ ] If possible, **restrict source IPs** (firewall / security group).  
- [ ] Prefer **SSH tunnels/VPN** (e.g., Tailscale, WireGuard) for private access instead of opening ports publicly.  
- [ ] If you need TLS quickly, place a lightweight **reverse proxy** (e.g., Caddy/Nginx) in front of the gateway to terminate HTTPS.  
- [ ] Monitor basic logs for anomalies while exposed.  
- [ ] Remember: H2 is **in‑memory** here — **data resets** on restart. Persistent DB comes in Module III.

---

## 6) Raspberry Pi (ARM) note — optional path later

If you later run the stack on a **Raspberry Pi (ARM64)**:

- Use a **64‑bit OS** if possible; ensure Java is available.  
- Either **build on the Pi** (native architecture) **or** use Docker **buildx** to produce ARM images.  
- Keep an eye on **memory** (tune JVM with `JAVA_OPTS`, e.g., `-Xmx256m` for small devices).  
- Network exposure guidance from the **Safe exposure checklist** still applies.

> For now, Phase 1 is about running the stack **on‑prem**. The Pi can act as a small remote node when you’re ready.

---

## 7) Observability & troubleshooting

- **Health**:  
  - Gateway → `GET /actuator/health`  
  - tasks‑service → `GET /actuator/health`  
- **Logs**:  
  - Compose → `docker compose logs -f <service>`  
  - STS → watch the Console tab

**Common issues**
- **401 from gateway**: `API_KEY` is enabled and `X-API-KEY` is missing/wrong.  
- **502/503 from gateway**: backend not reachable (check tasks‑service health).  
- **Port already in use**: stop old processes or change ports.  
- **Data “disappears”**: expected with **H2 in‑memory** (Phase 1 demo).

---

## 8) Tools used here (and why)

- **Spring Boot**: rapid API dev; embedded server; Actuator for health.  
- **Spring Cloud Gateway**: single entry point, simple policy (X‑API‑KEY), clean routing.  
- **Git**: phase **tags** keep the class synchronized on milestones.  
- **Docker Compose**: easy multi‑service bring‑up on a single host.  
- **STS**: great developer experience for local run/debug.  
- **Postman**: shareable requests, easier than `curl` for many attendees.

---

## ✅ Notebook 4 checkpoints

- [ ] I can bring up the **gateway (8080)** + **tasks‑service (8081)** stack with **Docker Compose** on an on‑prem host.  
- [ ] I validated `/actuator/health` on **both** services.  
- [ ] I tested routing via the gateway (`/api/tasks/**`) with **Postman**.  
- [ ] I enabled **X‑API‑KEY** (optional) and confirmed `401` without header, `200/201` with the correct header.  
- [ ] I understand the **risks of public exposure** in this MVP and basic mitigations.  
- [ ] I know that **persistence** (PostgreSQL) arrives in **Module III**.

---

## 🔜 After this

- Move to **Module III** to introduce a **managed Postgres** (or on‑prem Postgres) and proper environment configuration.  
- Later in **Module IV**, add **TLS**, **JWT/IAM**, **observability** (metrics/tracing), and **CI/CD**.
